<a href="https://colab.research.google.com/github/aganjasarthak/mnist/blob/main/train_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --quiet tensorflow tensorflowjs scikit-learn joblib

import os
import json
import numpy as np
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import StandardScaler
from google.colab import drive

drive.mount('/content/drive')
drive_path = "/content/drive/MyDrive/MNIST_CNN/"
os.makedirs(drive_path, exist_ok=True)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x = np.concatenate([x_train, x_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)
x = x.reshape(-1, 28, 28, 1).astype('float32') / 255.0

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()
model.fit(x, y, epochs=5, batch_size=128, validation_split=0.1)

saved_model_dir = os.path.join(drive_path, "tf_model_mnist")
tf.saved_model.save(model, saved_model_dir)
print(f"Saved TensorFlow SavedModel → {saved_model_dir}")

tfjs_target_dir = os.path.join(drive_path, "tfjs_model_mnist")
!tensorflowjs_converter \
  --input_format=tf_saved_model \
  --output_format=tfjs_graph_model \
  "{saved_model_dir}" "{tfjs_target_dir}"
print(f"Exported TFJS GraphModel → {tfjs_target_dir}")

preds = model.predict(x[:10])
print("sample preds:", np.argmax(preds, axis=1), " labels:", y[:10])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
493/493 ━━━━━━━━━━━━━━━━━━━━ 47s 92ms/step - accuracy: 0.8676 - loss: 0.4764 - val_accuracy: 0.9857 - val_loss: 0.0508
Epoch 2/5
493/493 ━━━━━━━━━━━━━━━━━━━━ 81s 91ms/step - accuracy: 0.9811 - loss: 0.0619 - val_accuracy: 0.9891 - val_loss: 0.0325
Epoch 3/5
493/493 ━━━━━━━━━━━━━━━━━━━━ 82s 91ms/step - accuracy: 0.9883 - loss: 0.0406 - val_accuracy: 0.9919 - val_loss: 0.0262
Epoch 4/5
493/493 ━━━━━━━━━━━━━━━━━━━━ 83s 94ms/step - accuracy: 0.9909 - loss: 0.0305 - val_accuracy: 0.9920 - val_loss: 0.0221
Epoch 5/5
493/493 ━━━━━━━━━━━━━━━━━━━━ 82s 94ms/step - accuracy: 0.9926 - loss: 0.0232 - val_accuracy: 0.9933 - val_loss: 0.0199
Saved TensorFlow SavedModel → /content/drive/MyDrive/MNIST_CNN/tf_model_mnist
2025-06-28 08:08:13.146862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751098093.170686    6547 cuda_dnn.cc:8310] Unable to r